In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

In [2]:
result = pd.read_csv('result.csv', index_col=0)
da = pd.read_csv('DaycaresAnnotations_original.csv')
result['detection'] = 'daycare'
da['detection'] = 'daycare'

In [3]:
result.head(5)

,file,conf,x,y,xmin,ymin,xmax,ymax,width,height,detection
0,11926_20161008.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
1,11926_20170331.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
2,11946_20140215.jpg,0.597667,747.607422,422.410187,670.337044,339.748055,824.8778,505.072319,154.540756,165.324265,daycare
3,12645_20160829.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
4,12650_20120928.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare


In [4]:
ground_truth = []
for r in range(len(da)):
    if da.loc[r, 'region_shape_attributes'] != "{}":
        filename = da.loc[r, 'filename']
        rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])
        # x-min
        xmin = int(rect['x'])
        # x-max
        xmax = int(rect['x'] + rect['width'])
        # y-min
        ymin = int(rect['y'])
        # y-max
        ymax = int(rect['y'] + rect['height'])
        width = rect['width']
        height = rect['height']
        detection = da.loc[r, 'detection']
        ground_truth.append([filename, detection, xmin, ymax, xmax, ymin, width, height])
        
ground_truth = pd.DataFrame(np.array(ground_truth), columns = ['file', 'detection', 'xmin', 'ymax', 'xmax', 'ymin', 'width', 'height'])
detection_result = result[['file', 'detection', 'conf', 'x', 'y']]
positive_detection_result = detection_result.loc[~((detection_result['x'] == 0) & (detection_result['y'] == 0))]
# detection_result = result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']]
# positive_detection_result = detection_result.loc[~((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]
# zero_detection_result = detection_result.loc[((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]


In [5]:
positive_detection_result.head(5)

,file,detection,conf,x,y
2,11946_20140215.jpg,daycare,0.597667,747.607422,422.410187
5,12753_20130505.jpg,daycare,0.948496,789.398987,455.202881
6,12753_20150725.jpg,daycare,0.962339,786.321045,455.010681
7,12753_20170712.jpg,daycare,0.763759,784.261719,455.274048
8,12753_20171004.jpg,daycare,0.667500,787.149963,459.464874


In [6]:
# ground_truth[['file', 'width', 'height']]
positive_detection_result = pd.merge(ground_truth[['file', 'width', 'height']], positive_detection_result, on='file', how='inner')


In [7]:
# positive_detection_result[['x', 'y']]

In [8]:
# def calculate_area(row):
#     rou(int(row['x']) - int(row['width']) /2 )
#     return row['x'] - row['width'] / 2
positive_detection_result[['x', 'y']] = positive_detection_result[['x', 'y']].astype(int)
positive_detection_result['xmin'] = positive_detection_result.apply(lambda row: int(row['x']) - round(int(row['width']) / 2), axis=1)
positive_detection_result['xmax'] = positive_detection_result.apply(lambda row: int(row['x']) + round(int(row['width']) / 2), axis=1)
positive_detection_result['ymin'] = positive_detection_result.apply(lambda row: int(row['y']) - round(int(row['height']) / 2), axis=1)
positive_detection_result['ymax'] = positive_detection_result.apply(lambda row: int(row['y']) + round(int(row['height']) / 2), axis=1)


positive_detection_result = positive_detection_result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']].drop_duplicates()


In [9]:
positive_detection_result.head(5)

,file,detection,conf,xmin,ymax,xmax,ymin
0,17364_20140412.jpg,daycare,0.507108,562,701,662,621
1,17471_20170623.jpg,daycare,0.576698,845,532,925,448
2,17483_20150408.jpg,daycare,0.873063,660,437,756,309
3,17483_20150524.jpg,daycare,0.777938,653,435,749,307
4,17483_20150612.jpg,daycare,0.557399,652,435,748,307


In [10]:
positive_detection_files = positive_detection_result['file'].values
# zero_detection_result_files = zero_detection_result['file'].values
relevant_ground_truth = ground_truth[ground_truth['file'].isin(positive_detection_files)].drop_duplicates()


In [11]:
print(len(relevant_ground_truth), len(positive_detection_result))
print(set(relevant_ground_truth['file'].values) - set(positive_detection_result['file'].values))  
assert len(relevant_ground_truth) == len(positive_detection_result)

297 297
set()


In [12]:
positive_detection_result[['xmin', 'ymax', 'xmax', 'ymin']] = positive_detection_result[['xmin', 'ymax', 'xmax', 'ymin']].astype(int)
positive_detection_result.sort_values(by=['file']).head(5)

,file,detection,conf,xmin,ymax,xmax,ymin
180,11946_20140215.jpg,daycare,0.597667,693,465,801,379
181,12753_20130505.jpg,daycare,0.948496,723,493,855,417
182,12753_20150725.jpg,daycare,0.962339,720,493,852,417
183,12753_20170712.jpg,daycare,0.763759,718,493,850,417
184,12753_20171004.jpg,daycare,0.667500,721,497,853,421


In [13]:
relevant_ground_truth = relevant_ground_truth[['file','detection','xmin','ymax','xmax','ymin']]
relevant_ground_truth.sort_values(by=['file']).head(5)

,file,detection,xmin,ymax,xmax,ymin
1542,11946_20140215.jpg,daycare,697,463,806,377
1564,12753_20130505.jpg,daycare,718,482,851,406
1569,12753_20150725.jpg,daycare,718,482,851,406
1577,12753_20170712.jpg,daycare,718,482,851,406
1579,12753_20171004.jpg,daycare,718,482,851,406


In [14]:
def put_files(df, loc):
    values = [[value[0], np.array2string(value[1:], separator=' ')
       .replace('[', '').replace(']', '').replace("'", "").replace("\n", "")
     ]for value in df.values]
    for value in values:
        filepath_label = loc + value[0]
        filepath_label = filepath_label.replace('jpg', 'txt')
#         print(value[1])
        with open(filepath_label, 'w') as f:
            f.write("%s\n" % value[1])
put_files(positive_detection_result, './mAP/input/detection-results/')
put_files(relevant_ground_truth, './mAP/input/ground-truth/')